<a href="https://colab.research.google.com/github/JinHuiXu1991/Jin_DATA606/blob/main/ipynb/DATA606_Part2_CollaborativeRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Product Recommender Systems
## Author: Jin Hui Xu

##Collaborative Filtering 

In [1]:
!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true

!wget https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_review.zip?raw=true

--2022-04-08 03:14:06--  https://github.com/JinHuiXu1991/Jin_DATA606/blob/main/cleaned_data/cleaned_amazon_product.zip?raw=true
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-04-08 03:14:06--  https://github.com/JinHuiXu1991/Jin_DATA606/raw/main/cleaned_data/cleaned_amazon_product.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip [following]
--2022-04-08 03:14:06--  https://raw.githubusercontent.com/JinHuiXu1991/Jin_DATA606/main/cleaned_data/cleaned_amazon_product.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133,

In [2]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 5.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630184 sha256=0ec6d481adeb954b57d3dfaaa0e0203362577a1225d3f2d6f3995edd780d0ec5
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split as train_test_split2
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from surprise import dump
import os
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise.model_selection import GridSearchCV

In [4]:
review_df = pd.read_csv('/content/cleaned_amazon_review.zip?raw=true', compression='zip')

review_df.head()

,overall,reviewTime,reviewerID,asin,reviewText,summary,reviewTimeYear,reviewTimeMonth
0,5.0,2013-11-27,A3NHUQ33CFH3VM,1118461304,not one thing in this book seemed an obvious o...,clear on what leads to innovation,2013,11
1,5.0,2013-11-01,A3SK6VNBQDNBJE,1118461304,i have enjoyed dr alan gregerman s weekly blog...,becoming more innovative by opening yourself t...,2013,11
2,5.0,2013-10-10,A3SOFHUR27FO3K,1118461304,alan gregerman believes that innovation comes ...,the world from different perspectives,2013,10
3,5.0,2013-10-09,A1HOG1PYCAE157,1118461304,alan gregerman is a smart funny entertaining w...,strangers are your new best friends,2013,10
4,5.0,2013-09-07,A26JGAM6GZMM4V,1118461304,as i began to read this book i was again remin...,how and why it is imperative to engage learn f...,2013,9


In [5]:
#Remove all information except Reviewer ID, Product ID and rating
review_df = review_df[['reviewerID', 'asin', 'overall']]

In [6]:
product_df = pd.read_csv('/content/cleaned_amazon_product.zip?raw=true', compression='zip')

product_df.head()

,category,description,title,brand,feature,main_cat,date,price,asin,imageURLHighRes,dateYear,dateMonth,ori_title
0,"Appliances Refrigerators, Freezers & Ice Makers",NaN,tupperware freezer square round container set ...,tupperware,each 3 pc set includes two 7 8 cup 200 ml and ...,appliances,2008-11-19,NaN,7301113188,[],2008,11,Tupperware Freezer Square Round Container Set ...
1,"Appliances Refrigerators, Freezers & Ice Makers",2 x tupperware pure fresh unique covered cool ...,2 x tupperware pure amp fresh unique covered c...,tupperware,2 x tupperware pure fresh unique covered cool ...,appliances,2016-06-05,3.62,7861850250,['https://images-na.ssl-images-amazon.com/imag...,2016,6,2 X Tupperware Pure &amp; Fresh Unique Covered...
2,Appliances Parts & Accessories,NaN,the cigar moments of pleasure,the cigar book,NaN,amazon home,NaN,150.26,8792559360,['https://images-na.ssl-images-amazon.com/imag...,0,0,The Cigar - Moments of Pleasure
3,Appliances Parts & Accessories,multi purpost descaler especially suited to wa...,caraselle 2x 50g appliance descalene,caraselle,NaN,tools home improvement,2014-12-17,NaN,9792954481,['https://images-na.ssl-images-amazon.com/imag...,2014,12,Caraselle 2X 50G Appliance Descalene
4,Appliances Parts & Accessories Range Parts & A...,full gauge and size beveled edge furnished wit...,eaton wiring 39ch sp l arrow hart 1 gang chrom...,eaton wiring,returns will not be honored on this closeout i...,tools home improvement,2007-01-16,3.43,B00002N5EL,[],2007,1,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...


In [7]:
#Remove all information except Product ID and title
product_df = product_df[['asin']]

In [8]:
merged_df = review_df.merge(product_df, on='asin', how='inner')
merged_df.head()

,reviewerID,asin,overall
0,A3SHVDMM83IHJ4,B00002N7IL,5.0
1,A3TIWHNJXMSIU7,B00002N7IL,5.0
2,AVP16JFIT6LPL,B00002N7IL,5.0
3,A2J2T9LX2HC947,B00002N7IL,5.0
4,AJN26F4KMWD59,B00002N7IL,5.0


In [9]:
merged_df = merged_df.groupby("reviewerID").filter(lambda x: len(x) > 2)

In [10]:
merged_df.shape

(49758, 3)

In [11]:
#Assign X as the original ratings dataframe and y as the user_id column of ratings.
X = merged_df.copy()
y = merged_df['reviewerID']

#Split into training and test datasets, stratified along user_id
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify=y, random_state=42)

In [12]:
#Function that computes the RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def baseline(reviewer_id, product_id):
    return 3.0

In [13]:
def score(cf_model):
    
    #Construct a list of user-movie tuples from the testing dataset
    id_pairs = zip(X_test['reviewerID'], X_test['asin'])
    
    #Predict the rating for every user-movie tuple
    y_pred = np.array([cf_model(reviewer, product) for (reviewer, product) in id_pairs])
    
    #Extract the actual ratings given by the users in the test data
    y_true = np.array(X_test['overall'])
    
    #Return the final RMSE score
    return rmse(y_true, y_pred)

In [14]:
score(baseline)

1.821168105646662

## Ratings Matrix

In [15]:
r_matrix = X_train.pivot_table(values='overall', index='reviewerID', columns='asin')

r_matrix

asin,B00002N7HY,B00002N7IL,B00002NARC,B00004SQHD,B00004YWK2,B00004YWK7,B00005OU6T,B00006IV17,B00007E7RQ,B00007E7RZ,...,B01HHFUBCC,B01HHFUCMG,B01HHFUCVW,B01HHFUD1G,B01HHFUDAC,B01HICGMWC,B01HIQV5DE,B01HJGWMTO,B01HJH0KLK,B01HJHHQM6
reviewerID,,,,,,,,,,,,,,,,,,,,,
A07936821FOVJO6NP4Q8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0954586PW98I9TNZXJP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1000EWBXBHK0P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100Q352397DAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100U4YIZTKJ56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZYLZ9TUBG4LR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZYW2VXM8Z2QH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZZHU3J9S09K4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#User Based Collaborative Filter using Mean Ratings
def cf_user_mean(reviewer_id, product_id):
    
    #Check if movie_id exists in r_matrix
    if product_id in r_matrix:
        #Compute the mean of all the ratings given to the movie
        mean_rating = r_matrix[product_id].mean()
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        mean_rating = 3.0
    
    return mean_rating

In [17]:
#Compute RMSE for the Mean model
score(cf_user_mean)

1.1634864033878112

## Weighted Mean

In [18]:
r_matrix_dummy = r_matrix.copy().fillna(0)

In [19]:
r_matrix_dummy

asin,B00002N7HY,B00002N7IL,B00002NARC,B00004SQHD,B00004YWK2,B00004YWK7,B00005OU6T,B00006IV17,B00007E7RQ,B00007E7RZ,...,B01HHFUBCC,B01HHFUCMG,B01HHFUCVW,B01HHFUD1G,B01HHFUDAC,B01HICGMWC,B01HIQV5DE,B01HJGWMTO,B01HJH0KLK,B01HJHHQM6
reviewerID,,,,,,,,,,,,,,,,,,,,,
A07936821FOVJO6NP4Q8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0954586PW98I9TNZXJP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1000EWBXBHK0P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100Q352397DAN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100U4YIZTKJ56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZYLZ9TUBG4LR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZYW2VXM8Z2QH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZZHU3J9S09K4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#Compute the cosine similarity matrix using the dummy ratings matrix
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

In [21]:
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim

reviewerID,A07936821FOVJO6NP4Q8,A0954586PW98I9TNZXJP,A1000EWBXBHK0P,A100Q352397DAN,A100U4YIZTKJ56,A100UD67AHFODS,A100WO06OQR8BQ,A101AF3PWTCCFZ,A102MK78XH84JN,A102VBZJRLHRHZ,...,AZVZXY07QD5D0,AZWOQXRCS1WA6,AZWVR4Y150V6L,AZXE7L2Q0E0CM,AZYHU1SCZT27T,AZYLZ9TUBG4LR,AZYW2VXM8Z2QH,AZZHU3J9S09K4,AZZRNGKUZKVIC,AZZV9PDNMCOZW
reviewerID,,,,,,,,,,,,,,,,,,,,,
A07936821FOVJO6NP4Q8,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A0954586PW98I9TNZXJP,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1000EWBXBHK0P,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100Q352397DAN,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100U4YIZTKJ56,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZYLZ9TUBG4LR,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
AZYW2VXM8Z2QH,0.0,0.0,0.0,0.0,0.0,0.0,0.161296,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
AZZHU3J9S09K4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [22]:
#User Based Collaborative Filter using Weighted Mean Ratings
def cf_user_wmean(reviewer_id, product_id):
    
    #Check if movie_id exists in r_matrix
    if product_id in r_matrix:
        
        #Get the similarity scores for the user in question with every other user
        sim_scores = cosine_sim[reviewer_id]
        
        #Get the user ratings for the product in question
        m_ratings = r_matrix[product_id]
        
        #Extract the indices containing NaN in the m_ratings series
        idx = m_ratings[m_ratings.isnull()].index
        
        #Drop the NaN values from the m_ratings Series
        m_ratings = m_ratings.dropna()
        
        #Drop the corresponding cosine scores from the sim_scores series
        sim_scores = sim_scores.drop(idx)
        
        #Compute the final weighted mean
        if sim_scores.sum() == 0.0:
          wmean_rating = 3.0
        else:
          wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
    
    else:
        #Default to a rating of 3.0 in the absence of any information
        wmean_rating = 3.0
    
    return wmean_rating

In [23]:
score(cf_user_wmean)

1.5632708277857408

## Model Based

In [40]:
merged_df2 = review_df.merge(product_df, on='asin', how='inner')
merged_df2 = merged_df2.groupby("reviewerID").filter(lambda x: len(x) > 2)
merged_df2.shape

(49758, 3)

In [41]:
benchmark = []
reader = Reader()
# Iterate over all algorithms
data = Dataset.load_from_df(merged_df2, reader)
algorithms = [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]

print ("Attempting: ", str(algorithms), '\n\n\n')

for algorithm in algorithms:
    print("Starting: " ,str(algorithm))
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=5, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    print("Done: " ,str(algorithm), "\n\n")

print ('\n\tDONE\n')

Attempting:  [<surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fba12724690>, <surprise.prediction_algorithms.matrix_factorization.SVDpp object at 0x7fba1248b2d0>, <surprise.prediction_algorithms.slope_one.SlopeOne object at 0x7fba1248b110>, <surprise.prediction_algorithms.matrix_factorization.NMF object at 0x7fba1248bd50>, <surprise.prediction_algorithms.random_pred.NormalPredictor object at 0x7fba1248b550>, <surprise.prediction_algorithms.knns.KNNBaseline object at 0x7fba1248bb10>, <surprise.prediction_algorithms.knns.KNNBasic object at 0x7fba03a7db90>, <surprise.prediction_algorithms.knns.KNNWithMeans object at 0x7fba1248dad0>, <surprise.prediction_algorithms.knns.KNNWithZScore object at 0x7fba1248da10>, <surprise.prediction_algorithms.baseline_only.BaselineOnly object at 0x7fba1248db50>, <surprise.prediction_algorithms.co_clustering.CoClustering object at 0x7fba1248d990>] 



Starting:  <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7fba127

In [42]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.945955,8.403479,0.343846
SVD,0.961418,2.428905,0.133167
BaselineOnly,0.983262,0.170621,0.060608
KNNBaseline,0.987308,6.693902,0.915389
KNNBasic,1.032494,6.285677,0.925558
KNNWithZScore,1.055964,7.172257,0.934571
KNNWithMeans,1.057424,6.805963,0.986298
SlopeOne,1.068968,2.633288,0.125322
CoClustering,1.070479,2.549475,0.059320


In [43]:
param_grid = {
    "n_epochs": [10, 20, 30, 40],
    "lr_all": [0.002, 0.005, 0.01],
    "reg_all": [0.02, 0.1]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], refit=True, cv=5, n_jobs=5)
gs.fit(data)

training_parameters = gs.best_params["rmse"]

print("BEST RMSE: \t", gs.best_score["rmse"])
print("BEST params: \t", gs.best_params["rmse"])

BEST RMSE: 	 0.9204622155764504
BEST params: 	 {'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.02}


In [44]:
# data = Dataset.load_from_df(merged_df2, reader)
# trainset, testset = train_test_split2(data, test_size=.2, random_state=42)

# # We'll use the famous SVD algorithm.
# algo = SVD(n_epochs = training_parameters['n_epochs'], lr_all = training_parameters['lr_all'], reg_all = training_parameters['reg_all'])

# cross_validate(algo, data, measures=['RMSE'], cv=10, verbose=True)

# # Train the algorithm on the trainset, and predict ratings for the testset
# algo.fit(trainset)
# predictions = algo.test(testset)

# # Then compute RMSE
# accuracy.rmse(predictions)

Evaluating RMSE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9303  0.9253  0.9193  0.9153  0.8892  0.9060  0.9008  0.9310  0.9249  0.8907  0.9133  0.0149  
Fit time          5.15    5.23    5.19    5.32    5.23    5.28    5.18    5.16    5.15    5.23    5.21    0.05    
Test time         0.04    0.04    0.04    0.05    0.04    0.07    0.04    0.04    0.04    0.04    0.05    0.01    
RMSE: 0.9072


0.9071653906409254

In [45]:
data = Dataset.load_from_df(merged_df2, reader)

# We'll use the famous SVD algorithm.
algo = SVD(n_epochs = training_parameters['n_epochs'], lr_all = training_parameters['lr_all'], reg_all = training_parameters['reg_all'])

cross_validate(algo, data, measures=['RMSE'], cv=10, verbose=True)

Evaluating RMSE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9213  0.9408  0.9021  0.9065  0.9076  0.9142  0.9264  0.8820  0.8892  0.9166  0.9107  0.0164  
Fit time          5.42    5.26    5.23    5.24    5.22    5.19    5.25    5.24    5.55    5.36    5.30    0.11    
Test time         0.04    0.04    0.04    0.05    0.05    0.04    0.09    0.04    0.05    0.05    0.05    0.01    


{'fit_time': (5.417150259017944,
  5.26068639755249,
  5.22632622718811,
  5.243238210678101,
  5.219182252883911,
  5.192458152770996,
  5.246567726135254,
  5.236577272415161,
  5.55486798286438,
  5.361462593078613),
 'test_rmse': array([0.92125335, 0.94081688, 0.90208136, 0.90650123, 0.9076411 ,
        0.9141526 , 0.92642145, 0.88195105, 0.88921723, 0.91655972]),
 'test_time': (0.04166388511657715,
  0.04176497459411621,
  0.04203200340270996,
  0.04526257514953613,
  0.052782535552978516,
  0.04018115997314453,
  0.0859992504119873,
  0.03971219062805176,
  0.04657173156738281,
  0.04686427116394043)}

In [46]:
trainset = data.build_full_trainset()
algo = SVD(n_epochs = training_parameters['n_epochs'], lr_all = training_parameters['lr_all'], reg_all = training_parameters['reg_all'])
algo.fit(trainset)

In [47]:
## SAVING TRAINED MODEL

model_filename = "./cr_model.pickle"

# Dump algorithm and reload it.
file_name = os.path.expanduser(model_filename)
dump.dump(file_name, algo=algo)


In [48]:
def load_model(model_filename):
    file_name = os.path.expanduser(model_filename)
    _, loaded_model = dump.load(file_name)
    return loaded_model

In [209]:
from collections import defaultdict
def collaborative_SVD_recommender(predictions, product_df):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, _, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:10]

    for uid, user_ratings in top_n.items():
      result = [iid for (iid, _) in user_ratings]

    return result, product_df[product_df['asin'].isin(result)]['ori_title'].tolist()

In [210]:
input_id = 'A8WEXFRWX1ZHH'
data1 = [input_id]
data2 = product_df['asin'].tolist()


df = pd.DataFrame(data1)
df.columns =['reviewerID']

df1 = pd.DataFrame(data2)
df1.columns =['asin']
# filter out reviewed products
reviewed_product = merged_df2[merged_df2['reviewerID'] == input_id].asin.unique().tolist()
df1 = df1[~df1['asin'].isin(reviewed_product)]

# Now to perform cross join, we will create
# a key column in both the DataFrames to
# merge on that key.
df['key'] = 1
df1['key'] = 1

# to obtain the cross join we will merge
# on the key and drop it.
result = pd.merge(df, df1, on ='key')

result['overall']=0.0
del result['key']
result


,reviewerID,asin,overall
0,A8WEXFRWX1ZHH,7301113188,0.0
1,A8WEXFRWX1ZHH,7861850250,0.0
2,A8WEXFRWX1ZHH,8792559360,0.0
3,A8WEXFRWX1ZHH,9792954481,0.0
4,A8WEXFRWX1ZHH,B00002N5EL,0.0
...,...,...,...
30230,A8WEXFRWX1ZHH,B01HJH92JQ,0.0
30231,A8WEXFRWX1ZHH,B01HJH2I5G,0.0
30232,A8WEXFRWX1ZHH,B01HJH0J1G,0.0
30233,A8WEXFRWX1ZHH,B01HJHHEA0,0.0


In [211]:
model_filename = "./cr_model.pickle"
loaded_model = load_model(model_filename)
valid_Dataset = Dataset.load_from_df(result, reader)

testset = [valid_Dataset.df.loc[i].to_list() for i in range(len(valid_Dataset.df))]
predictions = loaded_model.test(testset)

title_df = pd.read_csv('/content/cleaned_amazon_product.zip?raw=true', compression='zip')
asin, title = collaborative_SVD_recommender(predictions, title_df)

In [212]:
print('Collaborative Recommender Result for customer {}: '.format(input_id))
for i in range(0, 10):
  print('{}. {}, {}'.format(i+1, asin[i], title[i]))

Collaborative Recommender Result for customer A8WEXFRWX1ZHH: 
1. B00002N7HY, Leviton 5050 B01-0-000 Electrical Receptacle, 125/250 Vac, 50 A, 3 Pole, 3 Wire, Pack of 1, Black
2. B00009W3I4, HEARTLAND 21000 Energy Saving Dryer Vent Closure
3. B00009W3P8, Certified Appliance Accessories 3-Wire Closed-Eyelet 30-Amp Dryer Cord, 4ft
4. B00009W3PA, Certified Appliance Accessories 3-Wire Closed-Eyelet 30-Amp Dryer Cord, 6ft
5. B00009W3PL, Certified Appliance Accessories 3-Wire Closed-Eyelet 50-Amp Range Cord, 5ft
6. B0002YTM0I, Broan 413004 ADA Capable Non-Ducted Under-Cabinet Range Hood, 30-Inch, Stainless Steel
7. B0002Z1FF2, Eastman 60355 Washing Machine Discharge Hose
8. B0002Z1H5U, Eastman 48362 Polyethylene Tubing Icemaker Kit
9. B0006OQCLG, BestAir ES12 Kenmore 14911 / Emerson HDC-12 Replacement Wick Filter - 4 filters
10. B00076VESY, Emerson H06E-18 Universal Thermocouple, 18-inch


In [164]:
loaded_model.predict('A8WEXFRWX1ZHH','B00J4V77DO')

Prediction(uid='A8WEXFRWX1ZHH', iid='B00J4V77DO', r_ui=None, est=3.239316474739158, details={'was_impossible': False})

In [165]:
merged_df2[merged_df2['reviewerID']=='A8WEXFRWX1ZHH']

,reviewerID,asin,overall
69873,A8WEXFRWX1ZHH,B0014CN8Y8,5.0
71892,A8WEXFRWX1ZHH,B0014CN8Y8,5.0
99861,A8WEXFRWX1ZHH,B0006GVNOA,5.0
99871,A8WEXFRWX1ZHH,B0006GVNOA,5.0
99881,A8WEXFRWX1ZHH,B0006GVNOA,5.0
...,...,...,...
101871,A8WEXFRWX1ZHH,B0006GVNOA,5.0
101881,A8WEXFRWX1ZHH,B0006GVNOA,5.0
101891,A8WEXFRWX1ZHH,B0006GVNOA,5.0
570349,A8WEXFRWX1ZHH,B00IYZP4AY,4.0


In [168]:
merged_df2[merged_df2['asin']=='B00002N7HY']

,reviewerID,asin,overall
537508,A3TIWHNJXMSIU7,B00002N7HY,5.0
537509,A2UEH0U5CV3053,B00002N7HY,5.0
537511,AY41JC7MWFY1R,B00002N7HY,5.0
537513,ATCY52ZP70FKP,B00002N7HY,5.0
537514,A2V3DZW93FWH54,B00002N7HY,5.0
